In [11]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import re
import logging
import requests
from contextlib import contextmanager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException


def setup_logger():
    logger = logging.getLogger(__name__)
    
    # Nếu logger đã được cấu hình, trả về nó
    if logger.hasHandlers():
        return logger
    
    # Xóa tất cả handlers hiện có
    logger.handlers.clear()
    
    logger.setLevel(logging.INFO)
    
    # Tạo file handler
    file_handler = logging.FileHandler('scraper.log')
    file_handler.setLevel(logging.INFO)
    
    # Tạo console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    
    # Tạo formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    
    # Thêm handlers vào logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    # Ngăn chặn logger truyền messages lên các parent loggers
    logger.propagate = False
    
    return logger

# Xóa tất cả handlers của root logger
logging.getLogger().handlers.clear()
# Khởi tạo logger
logger = setup_logger()

@contextmanager
def managed_chrome_driver(webdriver_path):
    """
    Context manager để quản lý vòng đời của Chrome WebDriver.
    
    Args:
        webdriver_path (str): Đường dẫn đến ChromeDriver.
    
    Yields:
        webdriver.Chrome: Đối tượng WebDriver đã được khởi tạo.
    """
    options = Options()
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(webdriver_path), options=options)
    try:
        yield driver
    finally:
        driver.quit()

def scrape_page(url):
    """
    Cào dữ liệu từ một trang cụ thể.
    
    Args:
        url (str): URL của trang cần cào dữ liệu.
    
    Returns:
        tuple: (date, title, content) hoặc (None, None, None) nếu có lỗi.
    """
    try:
        with requests.Session() as session:
            page = session.get(url)
            page.raise_for_status()  # Kiểm tra lỗi HTTP
        
        soup = BeautifulSoup(page.text, 'html.parser')
        
        soup_title = soup.find('h1', class_="title")
        title = soup_title.text.strip() if soup_title else 'No Title'
        
        soup_content = soup.find_all('span', class_="richtext-text css-1iqe90x")
        content = ' '.join([c.text.strip() for c in soup_content])
        
        date_match = re.search(r'/(\d{4}-\d{2}-\d{2})-', url)
        date = date_match.group(1) if date_match else 'No Date'
        
        logger.info(f"Scraped URL: {url} | Date: {date} | Title: {title[:50]}...")
        
        return date, title, content
    except requests.RequestException as e:
        logger.error(f"Network error while scraping {url}: {e}")
    except AttributeError as e:
        logger.error(f"Parsing error while scraping {url}: {e}")
    except Exception as e:
        logger.error(f"Unexpected error while scraping {url}: {e}")
    
    return None, None, None

def get_all_post_urls(base_url, driver, target_date='2020-01-01'):
    """
    Lấy tất cả các URL bài viết bằng cách kéo xuống (scroll down) một cách tự nhiên.
    
    Args:
        base_url (str): URL cơ sở để bắt đầu cào dữ liệu.
        driver (webdriver.Chrome): Đối tượng WebDriver đã được khởi tạo.
        target_date (str): Ngày mục tiêu để dừng việc cào dữ liệu.
    
    Returns:
        list: Danh sách các URL bài viết.
    """
    post_urls = set()
    found_target_date = False
    scroll_pause_time = random.uniform(3, 7)
    last_height = driver.execute_script("return document.body.scrollHeight")
    no_new_content_count = 0
    max_no_new_content = 10

    try:
        driver.get(base_url)
        time.sleep(random.uniform(5, 10))  # Thời gian chờ ban đầu ngẫu nhiên

        while not found_target_date and no_new_content_count < max_no_new_content:
            # Scroll xuống với JavaScript
            scroll_height = random.randint(300, 700)  # Scroll một khoảng ngẫu nhiên
            driver.execute_script(f"window.scrollBy(0, {scroll_height});")
            time.sleep(random.uniform(0.5, 1.5))

            # Thỉnh thoảng thực hiện các hành động ngẫu nhiên
            if random.random() < 0.2:  # 20% cơ hội
                perform_random_action(driver)

            # Chờ đợi để trang tải thêm nội dung
            time.sleep(scroll_pause_time)

            # Mô phỏng hành vi đọc nội dung
            if random.random() < 0.3:  # 30% cơ hội dừng để "đọc"
                reading_time = random.uniform(5, 15)
                logger.info(f"Pausing to 'read' content for {reading_time:.2f} seconds")
                time.sleep(reading_time)

            # Tìm các liên kết mới
            new_urls_found = find_new_links(driver, post_urls, target_date)
            
            if new_urls_found:
                no_new_content_count = 0
            else:
                no_new_content_count += 1

            if found_target_date:
                break

            # Kiểm tra xem đã đến cuối trang chưa
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # Thử scroll thêm vài lần nữa để đảm bảo
                for _ in range(3):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(scroll_pause_time)
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height > last_height:
                        no_new_content_count = 0
                        break
                else:
                    if no_new_content_count >= max_no_new_content:
                        logger.info("Reached end of page or no new content. Refreshing...")
                        driver.refresh()
                        time.sleep(random.uniform(5, 10))
                        no_new_content_count = 0

            last_height = new_height

            # Cập nhật thời gian pause ngẫu nhiên
            scroll_pause_time = random.uniform(3, 7)

            logger.info(f"URLs found: {len(post_urls)}, Next wait time: {scroll_pause_time:.2f}s")

    except Exception as e:
        logger.error(f"Error during scrolling: {e}")

    logger.info(f"Total post URLs collected from {base_url}: {len(post_urls)}")
    return list(post_urls)

def perform_random_action(driver):
    """Thực hiện một hành động ngẫu nhiên trên trang."""
    try:
        actions = [
            lambda: driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP),
            lambda: driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN),
            lambda: driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.ARROW_UP),
            lambda: driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.ARROW_DOWN),
            lambda: WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.TAG_NAME, 'button'))).click()
        ]
        random.choice(actions)()
        logger.info("Performed a random action")
    except Exception as e:
        logger.warning(f"Failed to perform random action: {e}")

def find_new_links(driver, existing_urls, target_date):
    """Tìm các liên kết mới và thêm vào tập hợp existing_urls."""
    new_urls_found = False
    try:
        links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/en/square/post/')]"))
        )
        for link in links:
            url = link.get_attribute('href')
            if url not in existing_urls:
                date_match = re.search(r'/(\d{4}-\d{2}-\d{2})-', url)
                if date_match:
                    date = date_match.group(1)
                    if date <= target_date:
                        return True  # Tìm thấy target date
                    existing_urls.add(url)
                    new_urls_found = True
    except (TimeoutException, StaleElementReferenceException) as e:
        logger.warning(f"Error finding links: {e}")
    return new_urls_found

def process_url(url, webdriver_path):
    """
    Xử lý một URL cụ thể, cào dữ liệu và lưu vào file CSV.
    
    Args:
        url (str): URL cần xử lý.
        webdriver_path (str): Đường dẫn đến ChromeDriver.
    """
    with managed_chrome_driver(webdriver_path) as driver:
        try:
            post_urls = get_all_post_urls(url, driver, target_date='2020-01-01')
            
            data = []
            for i, post_url in enumerate(post_urls, 1):
                date, title, content = scrape_page(post_url)
                if date and title and content:
                    data.append({'Date': date, 'Title': title, 'Content': content})
                
                if i % 100 == 0:
                    partial_df = pd.DataFrame(data)
                    partial_filename = f'temp_{i}_posts.csv'
                    partial_df.to_csv(partial_filename, index=False, encoding='utf-8-sig')
                    logger.info(f"Temporary data saved to {partial_filename}")
            
            df = pd.DataFrame(data)
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.sort_values(by='Date', ascending=False).reset_index(drop=True)
            
            filename = 'bitcoin_news.csv' if 'bitcoin' in url else 'ethereum_news.csv' if 'ethereum' in url else 'output.csv'
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            logger.info(f"Data saved to {filename}")
        
        except Exception as e:
            logger.error(f"Error processing {url}: {e}")


webdriver_path = 'chromedriver.exe'
urls = [
    'https://www.binance.com/en/square/news/bitcoin%20news',
    'https://www.binance.com/en/square/news/ethereum%20news'
]
    
for url in urls:
    process_url(url, webdriver_path)

2024-06-25 07:41:57,476 - WARNING - Failed to perform random action: Message: element click intercepted: Element <button class="bn-button bn-button__primary data-size-large">...</button> is not clickable at point (1061, 24). Other element would receive the click: <div class="hidden-in-bnc-app css-pddht">...</div>
  (Session info: chrome=126.0.6478.114)
Stacktrace:
	GetHandleVerifier [0x00007FF6980B1F22+60322]
	(No symbol) [0x00007FF69802CE99]
	(No symbol) [0x00007FF697EE7EBA]
	(No symbol) [0x00007FF697F3F32E]
	(No symbol) [0x00007FF697F3CCF2]
	(No symbol) [0x00007FF697F3A18B]
	(No symbol) [0x00007FF697F39356]
	(No symbol) [0x00007FF697F2B491]
	(No symbol) [0x00007FF697F5C21A]
	(No symbol) [0x00007FF697F2ADB6]
	(No symbol) [0x00007FF697F5C430]
	(No symbol) [0x00007FF697F7BC80]
	(No symbol) [0x00007FF697F5BFC3]
	(No symbol) [0x00007FF697F29617]
	(No symbol) [0x00007FF697F2A211]
	GetHandleVerifier [0x00007FF6983C946D+3301613]
	GetHandleVerifier [0x00007FF698413693+3605267]
	GetHandleVerif

In [16]:
import pandas as pd

# Đọc dữ liệu từ file CSV vào DataFrame
df = pd.read_csv('bitcoin_news.csv')

# Chuyển cột 'Date' thành kiểu datetime để có thể so sánh
df['Date'] = pd.to_datetime(df['Date'])

# Lọc các bài viết từ ngày 2024-05-01 đến 2024-05-05
start_date = '2024-05-01'
end_date = '2024-05-05'
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# Hiển thị kết quả
print(filtered_df)


          Date                                              Title  \
252 2024-05-05  Bitcoin NFT Blob Team Announces Engraving of R...   
253 2024-05-05  Bitcoin (BTC) Surpasses 64,000 USDT with a 0.1...   
254 2024-05-05               Bitcoin Drops Below 63,000 USDT Mark   
255 2024-05-05  Bitcoin(BTC) Drops Below 63,000 USDT with a 0....   
256 2024-05-04  Block To Allocate 10% Of Bitcoin-Related Profi...   
257 2024-05-04  Bitcoin(BTC) Surpasses 63,000 USDT with a 5.93...   
258 2024-05-04  Marathon Digital Reports 21% Increase in BTC P...   
259 2024-05-03                 Binance Market Update (2024-05-03)   
260 2024-05-03  Bitcoin(BTC) Surpasses 62,000 USDT with a 5.01...   
261 2024-05-03  Elon Musk's Number Sequence Coincidentally Mat...   
262 2024-05-03  Bitcoin ETFs Decrease Holdings Amid Market Vol...   
263 2024-05-03  10xResearch: Bitcoin Investors Should Be Aware...   
264 2024-05-03  Hong Kong Bitcoin ETF Holds 4088 BTC After Two...   
265 2024-05-02  Bitcoin Layer2 Sol